In [2]:
import pandas as pd

In [4]:
df= pd.read_csv("user_rfm_data.csv")

In [5]:
df.head()

,User_id,Recency,Frequency,Monetary,R,F,M,Group,Score,Loyalty
0,51448162784,10,2,21500.0,4,4,4,444,12,Platinum
1,51455512201,64,1,10000.0,3,1,3,313,7,Gold
2,51462598385,7,2,550.0,4,4,1,441,9,Platinum
3,51463622150,40,2,950.0,4,4,1,441,9,Platinum
4,51465529647,10,2,15500.0,4,4,4,444,12,Platinum


In [36]:
df.DateTime.dtype

dtype('<M8[ns]')

In [34]:
df['DateTime'] = pd.to_datetime(df['DateTime'])


In [3]:
df.isna().sum()

User_id        0
Session_id     0
DateTime       0
Category       0
SubCategory    0
Action         0
Quantity       0
Rate           0
Total Price    0
dtype: int64

In [4]:
df['Action'].unique()

array(['first_app_open', 'search', 'product_view', 'read_reviews',
       'add_review', 'add_to_cart', 'checkout', 'purchase',
       'add_to_wishlist', 'click_wishlist_page', 'product_review'],
      dtype=object)

In [13]:
df["DateTime"] = pd.to_datetime(df["DateTime"])

ValueError: time data "31/10/2019 13:07:00" doesn't match format "%Y-%m-%d %H:%M:%S", at position 81. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [10]:
def get_rfm_features(data, cust_id="User_id", invoice_date="DateTime", total_sales="Total Price"):
    # creating a copy of the dataframe
    temp = data.copy()
    # converting the customer_id column to object type
    temp[cust_id] = temp[cust_id].astype("object")
   

    """ RECENCY"""
    # computing the last purchase feature of the users
    temp['LastPurchase'] = temp.groupby(cust_id)[invoice_date].transform(lambda x: x.max(skipna=True))
    # getting the last invoice
    last_invoice = temp[invoice_date].max() + pd.Timedelta(days=1)
    # compuint the recency feature
    temp['Recency'] = (last_invoice-temp['LastPurchase']).dt.days
    # creating a separate dataframe which consists receny for each users
    users_receny = temp[[cust_id,"Recency"]].drop_duplicates().sort_values(by=cust_id).set_index(cust_id)


    """ FREQUENCY """
    # creating a df which contains frequency features for each users
    users_frequency = pd.DataFrame(temp[cust_id].value_counts().sort_index()).rename(columns={"count":"Frequency"})

    """ MONETARY """
    users_monetary = pd.DataFrame(temp.groupby(cust_id)[total_sales].sum().sort_index()).rename(columns={total_sales: "Monetary"})

    """ RFM Dataframe"""
    # Joining Recency and Frequency DataFrames
    RFM = pd.merge(users_receny, users_frequency, left_index=True, right_index=True)
    # Joining with Monetary DataFrame
    RFM = pd.merge(RFM, users_monetary, left_index=True, right_index=True).reset_index()

    return RFM

In [11]:
RFM = get_rfm_features(data=df[df["Action"]=="purchase"],cust_id="User_id",invoice_date="DateTime",total_sales="Total Price")

ValueError: time data "31/10/2019 15:28:00" doesn't match format "%Y-%m-%d %H:%M:%S", at position 5. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.